<a href="https://colab.research.google.com/github/neiltheblue/loial/blob/main/Loial_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Loial pacakge needs to be available, as package or from the git repo.

In [74]:
!pip install -i https://test.pypi.org/simple/ loial

Looking in indexes: https://test.pypi.org/simple/


# Simple Python builder example

The current Loial project supports two buildedrs. One is the python builder that is used mainly to exercise the framework, and the CC compiler that is used to integrate compiles libraries. So far this has only been tested using Linux.

This first exmaple shows how a python method can be replaced by an alternative implemention.

In [75]:
def grow(x):
  return x + x

assert grow(3) == 6

The alternative implementation. Here the content of the build decorator is used instead of the python method body.

In [76]:
from loial import build

@build('return x ** 2')
def grow(x):
  return x + x

assert grow(3) == 9

This alternative behaviour can be turned off.

In [77]:
from loial import build

@build('return x ** 2', replace=False)
def grow(x):
  return x + x

assert grow(3) == 6

If the alternative code does not compile, then the original body will not be replaced,

In [78]:
from loial import build

@build('return x to the power of 2')
def grow(x):
  return x + x

assert grow(3) == 6
print(f'Output: {grow(3)}')

ERROR:loial.builders.python_builder:Error compiling code, using defaut: invalid syntax (<string>, line 1)


Output: 6


# C builder example

These examples demonstrate how to compile C code to replace python functions.

This first example shows how to call a simple that take no arguments and returns an `int` value.

Note that this uses the `@cc_build` decorator, and unless decfined otherwise all values are assumed to be `int`.

Unlike with the python builder, the C code must include a function, which by default has the same name as the python function that is being replaced.

In [79]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun1() {
    return 10;
}
''')
def fun1():
    return 1

assert fun1() == 10

This next example passes in an int argument from python in to the C function and returns an `int` value

In [80]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun2(int a) {
    return a * 10;
}
''')
def fun2(a):
    return a

assert fun2(3) == 30

Multiple arguments are also supported.

In [81]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun3(int a, int b, int c) {
    return a + b + c;
}
''')
def fun3(a, b, c):
    return a*b*c

assert fun3(1, 2, 3) == 6

Calling the replaced python function also supports named arguments.

Here the arguments `d` & `e` are names and used in reverse order.

In [82]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun4(int a, int b, int c, int d, int e) {
    return ((a + b + c)*d)/e;
}
''')
def fun4(a, b, c, d, e):
    return (a*b*c+d)*e

assert fun4(1, 2, 3, e=2, d=10) == 30

Default python values are also supported when calling a replacement C function.

Here the default value for `e` is being used in the method call

In [83]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun5(int a, int b, int c, int d, int e) {
    return ((a + b + c)*d)/e;
}
''')
def fun5(a, b, c, d, e=2):
    return (a*b*c+d)*e

assert fun5(1, 2, 3, d=10) == 30

If there is an error or the C code does not compile, then the default python implementation is used as a fallback.

However the compile failure is logged using standard python logging.

In [84]:
from loial.builders.cc_builder import cc_build

@cc_build('''
    junk
''')
def bad(a, b, c, d):
    return (a+b+c)*d

assert bad(1, 2, 3, 10) == 60

print(f'{"="*10}\nSuccessful result {bad(1, 2, 3, 10)}')

ERROR:loial.builders.cc_builder:Error compiling code: /content/mycache/lib__main__.bad_61337cf369816ff5d971d30baf628637.so.c:2:5: error: expected ‘=’, ‘,’, ‘;’, ‘asm’ or ‘__attribute__’ at end of input
    2 |     junk
      |     ^~~~
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/loial/builders/cc_builder.py", line 356, in cc_compile
    out = subprocess.run(
          ^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['cc', '-I', '/tmp', '-fPIC', '-shared', '-o', '/content/mycache/lib__main__.bad_61337cf369816ff5d971d30baf628637.so', '/content/mycache/lib__main__.bad_61337cf369816ff5d971d30baf628637.so.c']' returned non-zero exit status 1.


Successful result 60


When building a replacement function, an extra configuraiton instance can be passed in.

This config can be used to auto delete the compiled shared object when the application exits.

When the function is replaced by the C code, the new function instance can be used to access the new callable replacement.

By default the `CC_Builder` will look through the search path to find a location to cache the compiled library.

In [85]:
from loial.builders.cc_builder import CC_Config

print(CC_Config().cache_search_path)

['./mycache']


In [86]:
!rm -rf /root/.loial

In [87]:
from loial.builders.cc_builder import cc_build, CC_Config

@cc_build('''
int delete_me() {
    return 10;
}
''', CC_Config(delete_on_exit=True))
def delete_me():
    return 1

assert delete_me(0) == 10

print(delete_me.callable.so_file)


/content/mycache/lib__main__.delete_me_e58d682f73d55cd5c53a8970829516b4.so


In [88]:
!ls /root/.loial*

ls: cannot access '/root/.loial*': No such file or directory


Alternatively the global value can be set in the `CC_Config` class

In [89]:
from loial.builders.cc_builder import cc_build, CC_Config

CC_Config.delete_on_exit=True

@cc_build('''
int delete_me() {
    return 10;
}
''')
def delete_me():
    return 1

assert delete_me(0) == 10

print(delete_me.callable.so_file)

/content/mycache/lib__main__.delete_me_e58d682f73d55cd5c53a8970829516b4.so


In [90]:
!ls /root/.loial*

ls: cannot access '/root/.loial*': No such file or directory


The default search path to find a cache directory can be configured in the `CC_Config` class.

In [91]:
from loial.builders.cc_builder import cc_build, CC_Config

CC_Config.cache_search_path = [f'./mycache']

@cc_build('''
int cached() {
    return 10;
}
''')
def cached():
    return 1

assert cached(0) == 10

print(cached.callable.so_file)

/content/mycache/lib__main__.cached_3acec0a97280c767692df798daf4a79e.so


C typed parameters can be used if defined as hints in the python function defintion.

Here the standard `ctypes` are being used, and they also support named and default values.

Also in this example the standard io library is also included.

In [92]:
import ctypes
from loial.builders.cc_builder import cc_build

@cc_build(r'''
#include <stdio.h>
int typed(short a, float b, float c) {

    printf("sizes:%d %d %d\n", sizeof(a), sizeof(b), sizeof(c));
    printf("values:%d %d %d\n", a, b, c);
    return a + b + c;
}
''')
def typed(a: ctypes.c_short, b: ctypes.c_float, c: ctypes.c_float, d: ctypes.c_float = 2.0):
    return a*b*c

assert typed(1, 2.0, c=3.0) == 6

The return type can also be defined using hints.

In [93]:
from loial.builders.cc_builder import cc_build

@cc_build(r'''
float freturn(int a, int b, int c) {
    return (a + b + c)/2.0;
}
''')
def freturn(a, b, c) -> ctypes.c_float:
    return a*b*c

assert freturn(1, 2, 3) == 3.0

Using the CC_Config, the default `cc` compiler can also be configured.

In [94]:
from loial.builders.cc_builder import cc_build

@cc_build(r'''
int comp(int a, int b, int c) {
    return a + b + c;
}
''', CC_Config(compiler='gcc'))
def comp(a, b, c):
    return a*b*c

assert comp(1, 2, 3) == 6

The compiler options can also be updated.

In [95]:
from loial.builders.cc_builder import cc_build, CC_Config

print(CC_Config().compiler_opts)

@cc_build(r'''
int comp(int a, int b, int c) {
    return a + b + c;
}
''', CC_Config(compiler_opts=('-fPIC', '-shared', '-O3')))
def comp(a, b, c):
    return a*b*c

assert comp(1, 2, 3) == 6
print(comp.callable.config.compiler_opts)

('-fPIC', '-shared')
('-fPIC', '-shared', '-O3')


The config object can also be used to replace the C function name that is called.

In [96]:
from loial.builders.cc_builder import cc_build, CC_Config

@cc_build(r'''
int main(int a, int b, int c) {
    return a + b + c;
}
''', CC_Config(function='main'))
def not_main(a, b, c):
    return a*b*c

assert not_main(1, 2, 3) == 6

Values can be passed into the C function as references if they have a hint defined type.

In [97]:
import ctypes
from loial.builders.cc_builder import cc_build, AsRef

@cc_build('''
int ref(int* a, int b) {
    return *a * b;
}
''')
def ref(a: ctypes.c_int, b):
    return a+b

a = 3
b = 10
assert ref(AsRef(a), b) == 30

Or the parameter name be registered in the config to pass by ref.

In [98]:
import ctypes
from loial.builders.cc_builder import cc_build, CC_Config

@cc_build('''
int ref(int* a, int b) {
    return *a * b;
}
''', CC_Config(refs={'a'}))
def ref(a: ctypes.c_int, b):
    return a+b

a = 3
b = 10
assert ref(a, b) == 30

The use of pass by ref is more efficient, but if passing by pointer is needed then it is possible in a simlar way.

In [99]:
import ctypes
from loial.builders.cc_builder import cc_build, AsPointer

@cc_build('''
int ptr(int* a, int b) {
    *a=99;
    return *a * b;
}
''')
def ptr(a: ctypes.c_int, b):
    return a+b

a = AsPointer(3)
b = 10
assert ptr(a, b) == 990
assert a.value == 99

An array can be passed in as a list if the array type is defined as a parameter hint.

In [100]:
import ctypes
from loial.builders.cc_builder import cc_build

@cc_build('''
int arr(int a[], int b) {
    int i;
    int sum=0;
    for(i=0; i<b; i++)
    {
        sum = sum + a[i];
    }
    return sum;
}
''')
def arr(a: ctypes.c_int, b):
    return max(a)

assert arr([1, 2, 3], 3) == 6

Python classes can be passed in a structures by extending `C_Struct` and adding attribute hints. The `C_Struct` class also defines a method `define` to output a C structure for the class.

In [106]:
import ctypes
from loial.builders.cc_builder import cc_build, C_Struct

class Field(C_Struct):
    _fields_ = [("a", ctypes.c_int),
                ("b", ctypes.c_int)]

class Record(C_Struct):
    _fields_ = [("first", ctypes.c_int),
                ("second", ctypes.c_bool),
                ("third", ctypes.c_char),
                ("fourth", ctypes.c_wchar),
                ("fifth", ctypes.c_byte),
                ("sixth", ctypes.c_ubyte),
                ("seventh", ctypes.c_short),
                ("eighth", ctypes.c_ushort),
                ("nineth", ctypes.c_int),
                ("tenth", ctypes.c_uint),
                ("eleventh", ctypes.c_long),
                ("twelth", ctypes.c_ulong),
                ("thirteenth", ctypes.c_longlong),
                ("fourteenth", ctypes.c_ulonglong),
                ("fifteenth", ctypes.c_size_t),
                ("sixteenth", ctypes.c_ssize_t),
                ("seventeenth", ctypes.c_float),
                ("eighteenth", ctypes.c_double),
                ("nineteenth", ctypes.c_longdouble),
                ("twenty", ctypes.c_char_p),
                ("twentyone", ctypes.c_wchar_p),
                ("twentytwo", ctypes.c_void_p),
                ("field", Field),
                ]

@cc_build('''
#include<stdio.h>
#include <wchar.h>
          '''
          + Field.define()
          + Record.define() +
          r'''

int stru(Record r) {
    printf("first:%d (%lu)\n", r.first, sizeof(r.first));
    printf("second:%d (%lu)\n", r.second, sizeof(r.second));
    printf("third:%c (%lu)\n", r.third, sizeof(r.third));
    printf("fourth:%ld (%lu)\n", r.fourth, sizeof(r.fourth));
    printf("fifth:%d (%lu)\n", r.fifth, sizeof(r.fifth));
    printf("sixth:%d (%lu)\n", r.sixth, sizeof(r.sixth));
    printf("seventh:%d (%lu)\n", r.seventh, sizeof(r.seventh));
    printf("eighth:%d (%lu)\n", r.eighth, sizeof(r.eighth));
    printf("nineth:%d (%lu)\n", r.nineth, sizeof(r.nineth));
    printf("tenth:%u (%lu)\n", r.tenth, sizeof(r.tenth));
    printf("eleventh:%ld (%lu)\n", r.eleventh, sizeof(r.eleventh));
    printf("twelth:%lu (%lu)\n", r.twelth, sizeof(r.twelth));
    printf("thirteenth:%lld (%lu)\n", r.tenth, sizeof(r.thirteenth));
    printf("fourteenth:%llu (%lu)\n", r.fourteenth, sizeof(r.fourteenth));

    printf("fifteenth:%zx (%lu)\n", r.fifteenth, sizeof(r.fifteenth));
    printf("sixteenth:%zx (%lu)\n", r.sixteenth, sizeof(r.sixteenth));
    printf("seventeenth:%f (%lu)\n", r.seventeenth, sizeof(r.seventeenth));
    printf("eighteenth:%lf (%lu)\n", r.eighteenth, sizeof(r.eighteenth));
    printf("nineteenth:%llf (%lu)\n", r.nineteenth, sizeof(r.nineteenth));
    printf("twenty:%c (%lu)\n", *r.twenty, sizeof(r.twenty));
    printf("twentyone:%ld (%lu)\n", *r.twentyone, sizeof(r.twentyone));
    printf("twentytwo:%lu (%lu)\n", r.twentytwo, sizeof(r.twentytwo));
    printf("field:%d %d\n", r.field.a, r.field.b);
    return r.field.a * r.field.b;
}
''')
def stru(dom):
    return str(dom)

dom = Record()
dom.first = 3
dom.second = True
dom.third = b'a'
dom.fourth = 'b'
dom.fifth = 127
dom.sixth = 255
dom.seventh = -1
dom.eighth = -1
dom.nineth = -2
dom.tenth = -2
dom.eleventh = -3
dom.twelth = -3
dom.thirteenth = -4
dom.fourteenth = -4
dom.fifteenth = 123
dom.sixteenth = 124
dom.seventeenth = 1.23
dom.eighteenth = 4.32
dom.nineteenth = 2.34
dom.twenty = b'c'
dom.twentyone = 'd'
dom.twentytwo = 999
field = Field()
field.a = 2
field.b = 10
dom.field = field

assert stru(dom) == 20

As an alternative to the explicit class definition, the `c_struct` class decorator can be used.

In [105]:
import ctypes
from loial.builders.cc_builder import cc_build, c_struct


class SuperClass():
    pass

@c_struct
class LocalInnerStruct(SuperClass):
    c: ctypes.c_short

@c_struct
class LocalStruct():
    a: ctypes.c_int
    b: ctypes.c_float
    multi: LocalInnerStruct

@cc_build('''
#include<stdio.h>
          '''
          + LocalInnerStruct.define()
          + LocalStruct.define() +
          r'''

int loca(LocalStruct s) {
    printf("a:%d (%lu)\n", s.a, sizeof(s.a));
    printf("b:%f (%lu)\n", s.b, sizeof(s.b));
    return (s.a * s.b) * s.multi.c;
}
''')
def loca(loc) -> ctypes.c_float:
    return str(loc)

loc = LocalStruct()
loc.a = 3
loc.b = 3.5
loc.multi = LocalInnerStruct()
loc.multi.c = 5

assert loca(loc) == 52.5

External header files can be included in the compilation.

In [119]:
!mkdir -p header
!echo "" > header/values.h
!echo "/*" >> header/values.h
!echo " * test include header file" >> header/values.h
!echo " */" >> header/values.h
!echo "int VALUE = 2;" >> header/values.h
!cat header/values.h



/*
 * test include header file
 */
int VALUE = 2;


In [124]:
from loial.builders.cc_builder import cc_build

@cc_build('''
#include "values.h"

int inc1(int a) {
    return a * VALUE;
}
''', CC_Config(includes=['header']))
def inc1(a):
    return a

assert inc1(3) == 6

Multiple C source files may be included in the compilation if defined in the config.

In [125]:
from loial.builders.cc_builder import cc_build

src_file2 = CC_Config().create_cache_path('src2.c')
with open(src_file2, 'w') as out:
    out.write('''
              int ten(int a);

              int ten(int a)
              {
                  return a * 10;
              }
              ''')

src_file3 = CC_Config().create_cache_path('src3.c')
with open(src_file3, 'w') as out:
    out.write('''
              int dub(int a);

              int dub(int a)
              {
                  return a * 2;
              }
              ''')

@cc_build('''
int ten(int a);
int dub(int a);

int src1(int a) {
    return dub(ten(a));
}
''', CC_Config(src_files=[src_file2, src_file3]))
def src1(a):
    return a

assert src1(3) == 60

They can even be used to replace the code definition.

In [126]:
from loial.builders.cc_builder import cc_build, CC_Config

src_file1 = CC_Config().create_cache_path('src1.c')
with open(src_file1, 'w') as out:
    out.write('''
                int ten(int a);
                int dub(int a);

                int src1(int a) {
                    return dub(ten(a));
                }
              ''')

src_file2 = CC_Config().create_cache_path('src2.c')
with open(src_file2, 'w') as out:
    out.write('''
              int ten(int a);

              int ten(int a)
              {
                  return a * 10;
              }
              ''')

src_file3 = CC_Config().create_cache_path('src3.c')
with open(src_file3, 'w') as out:
    out.write('''
              int dub(int a);

              int dub(int a)
              {
                  return a * 2;
              }
              ''')

@cc_build(config=CC_Config(src_files=[src_file1, src_file2, src_file3]))
def src1(a):
    return a

assert src1(3) == 60

Object files may also be included in the source list.

Using the `CC_Builder.cc_compile_obj` helper method

In [129]:
from loial.builders.cc_builder import cc_build, CC_Config, CC_Builder


src_file1 = CC_Config().create_cache_path('src1.c')
with open(src_file1, 'w') as out:
    out.write('''
                int ten(int a);
                int dub(int a);

                int src1(int a) {
                    return dub(ten(a));
                }
              ''')

src_file2 = CC_Config().create_cache_path('src2.c')
with open(src_file2, 'w') as out:
    out.write('''
              int ten(int a);

              int ten(int a)
              {
                  return a * 10;
              }
              ''')

obj_file = CC_Builder.cc_compile_obj('''
                  int dub(int a);

                  int dub(int a)
                  {
                      return a * 2;
                  }
                  ''', CC_Config().create_cache_path('src3.o'))

@cc_build(config=CC_Config(src_files=[obj_file, src_file1, src_file2]))
def src1(a):
    return a

assert src1(3) == 60

Static libraries may be linked in, and can be built with the helper method `CC_Builder.archive`.

In [130]:
from loial.builders.cc_builder import cc_build, CC_Config, CC_Builder

obj_file = CC_Builder.cc_compile_obj('''
              int dub(int a);

              int dub(int a)
              {
                  return a * 2;
              }
              ''', CC_Config().create_cache_path('srclib.o'))

lib_file = CC_Builder.archive(
    CC_Config().create_cache_path('srclib.a'), [obj_file])

@cc_build('''
        int dub(int a);

                int src1(int a) {
                    return dub(a*10);
                }
              ''', CC_Config(static_libs=[lib_file]))
def src1(a):
    return a

assert src1(3) == 60

Shared libraries may be included from the library path using their short name.

In [131]:
from loial.builders.cc_builder import cc_build

@cc_build('''
        int work(int a) {
            return a-2;
        }
        ''', CC_Config(shared_libs=['stdc++']))
def work(a):
    return a+1

assert work(3) == 1

Callback functions may be passed to the C code using an explicit definition.

In [132]:
from loial.builders.cc_builder import cc_build

@ctypes.CFUNCTYPE(ctypes.c_float, ctypes.c_int, ctypes.c_int)
def cb(a, b):
    return (a+b)*10.0

@cc_build('''
        float cbfun(int a, int b, float (*cb)(int, int)) {
            return cb(a,b);
        }
        ''')
def cbfun(a, b, cb)->ctypes.c_float:
    return a-b

assert cbfun(1, 1, cb) == 20

The callback may be used implicitly if the paramter types are defined as hints.

In [133]:
import ctypes
from loial.builders.cc_builder import cc_build

def cb(a: ctypes.c_int, b: ctypes.c_int) ->ctypes.c_float:
    return (a+b)*10.0

@cc_build('''
        float cbfun(int a, int b, float (*cb)(int, int)) {
            return cb(a,b);
        }
        ''')
def cbfun(a, b, cb) -> ctypes.c_float:
    return a-b

assert cbfun(1, 1, cb) == 20